# W_GL_BALANCE_F ETL Process
### Staging to Fact - Complete Dimension Lookups and Incremental Update
**ODI Package:** SILOS_SIL_GLBALANCEFACT  
**IKM:** BIAPPS Oracle Fact Incremental Update  
**Detection Strategy:** OUTER with change detection on timestamp columns  
**Flow Table:** I$_2625555_5 with 90 columns and comprehensive dimension lookups

In [ ]:
%sql
-- Step 1: Create Flow Table I$_2625555_5 with ALL 90 columns and dimension lookups
-- This replicates the ODI flow logic with all dimension WID lookups
DROP TABLE IF EXISTS I$_2625555_5;

CREATE OR REPLACE TEMP VIEW I$_2625555_5 AS
SELECT
  -- Dimension WID Lookups (using COALESCE for 0 default)
  COALESCE(LKP_LEDGER.ROW_WID, 0) AS LEDGER_WID,
  COALESCE(LKP_COST_CENTER.ROW_WID, 0) AS COST_CENTER_WID,
  COALESCE(LKP_BALANCING_SEG.ROW_WID, 0) AS BALANCING_SEGMENT_WID,
  COALESCE(LKP_NATURAL_ACCT.ROW_WID, 0) AS NATURAL_ACCOUNT_WID,
  COALESCE(LKP_COMPANY_ORG.ROW_WID, 0) AS COMPANY_ORG_WID,
  COALESCE(LKP_BUSN_AREA_ORG.ROW_WID, 0) AS BUSN_AREA_ORG_WID,
  COALESCE(LKP_GL_ACCOUNT.ROW_WID, 0) AS GL_ACCOUNT_WID,
  COALESCE(LKP_BALANCE_DT.ROW_WID, 0) AS BALANCE_DT_WID,
  COALESCE(LKP_BALANCE_TM.ROW_WID, 0) AS BALANCE_TM_WID,
  COALESCE(LKP_TREASURY_SYMBOL.ROW_WID, 0) AS TREASURY_SYMBOL_WID,
  COALESCE(LKP_MCAL_CAL.ROW_WID, 0) AS MCAL_CAL_WID,
  COALESCE(LKP_GL_DATASET_SEC.ROW_WID, 0) AS GL_DATASET_SEC_WID,
  
  -- Fact Measures and Attributes from W_GL_BALANCE_FS
  FS.DB_CR_IND,
  FS.BALANCE_ACCT_AMT,
  COALESCE(FS.BALANCE_LOC_AMT, 0) AS BALANCE_LOC_AMT,
  COALESCE(FS.BALANCE_ACCT_AMT * EXCH.GLBL1_EXCH_RATE, 0) AS BALANCE_GLOBAL1_AMT,
  COALESCE(FS.BALANCE_ACCT_AMT * EXCH.GLBL2_EXCH_RATE, 0) AS BALANCE_GLOBAL2_AMT,
  COALESCE(FS.BALANCE_ACCT_AMT * EXCH.GLBL3_EXCH_RATE, 0) AS BALANCE_GLOBAL3_AMT,
  FS.ACTIVITY_ACCT_AMT,
  COALESCE(FS.ACTIVITY_LOC_AMT, 0) AS ACTIVITY_LOC_AMT,
  COALESCE(FS.ACTIVITY_ACCT_AMT * EXCH.GLBL1_EXCH_RATE, 0) AS ACTIVITY_GLOBAL1_AMT,
  COALESCE(FS.ACTIVITY_ACCT_AMT * EXCH.GLBL2_EXCH_RATE, 0) AS ACTIVITY_GLOBAL2_AMT,
  COALESCE(FS.ACTIVITY_ACCT_AMT * EXCH.GLBL3_EXCH_RATE, 0) AS ACTIVITY_GLOBAL3_AMT,
  FS.ACCT_CURR_CODE,
  FS.LOC_CURR_CODE,
  
  -- Integration Keys
  FS.INTEGRATION_ID,
  FS.DATASOURCE_NUM_ID,
  FS.TENANT_ID,
  FS.X_CUSTOM,
  
  -- Flags
  FS.BALANCE_TYPE_FLAG,
  FS.TRANSLATED_FLAG,
  FS.SUMMARY_ACCOUNT_FLAG,
  
  -- Timestamps
  FS.CREATED_ON_DT,
  FS.CHANGED_ON_DT,
  FS.AUX1_CHANGED_ON_DT,
  FS.AUX2_CHANGED_ON_DT,
  FS.AUX3_CHANGED_ON_DT,
  FS.AUX4_CHANGED_ON_DT,
  
  -- GL Segment WIDs (1-10) from W_GL_SEGMENT_D
  COALESCE(LKP_GL_SEG1.ROW_WID, 0) AS GL_SEGMENT1_WID,
  COALESCE(LKP_GL_SEG2.ROW_WID, 0) AS GL_SEGMENT2_WID,
  COALESCE(LKP_GL_SEG3.ROW_WID, 0) AS GL_SEGMENT3_WID,
  COALESCE(LKP_GL_SEG4.ROW_WID, 0) AS GL_SEGMENT4_WID,
  COALESCE(LKP_GL_SEG5.ROW_WID, 0) AS GL_SEGMENT5_WID,
  COALESCE(LKP_GL_SEG6.ROW_WID, 0) AS GL_SEGMENT6_WID,
  COALESCE(LKP_GL_SEG7.ROW_WID, 0) AS GL_SEGMENT7_WID,
  COALESCE(LKP_GL_SEG8.ROW_WID, 0) AS GL_SEGMENT8_WID,
  COALESCE(LKP_GL_SEG9.ROW_WID, 0) AS GL_SEGMENT9_WID,
  COALESCE(LKP_GL_SEG10.ROW_WID, 0) AS GL_SEGMENT10_WID,
  
  -- Budget Related WIDs
  COALESCE(LKP_BUDGET_PERIOD.ROW_WID, 0) AS BUDGET_PERIOD_WID,
  COALESCE(LKP_BALANCE_TYPE.ROW_WID, 0) AS BALANCE_TYPE_WID,
  
  -- Carry Forward Amounts
  COALESCE(FS.CARRY_FORWARD_ACCT_AMT, 0) AS CARRY_FORWARD_ACCT_AMT,
  COALESCE(FS.CARRY_FORWARD_LOC_AMT, 0) AS CARRY_FORWARD_LOC_AMT,
  COALESCE(FS.CARRY_FORWARD_ACCT_AMT * EXCH.GLBL1_EXCH_RATE, 0) AS CARRY_FORWARD_GLOBAL1_AMT,
  COALESCE(FS.CARRY_FORWARD_ACCT_AMT * EXCH.GLBL2_EXCH_RATE, 0) AS CARRY_FORWARD_GLOBAL2_AMT,
  COALESCE(FS.CARRY_FORWARD_ACCT_AMT * EXCH.GLBL3_EXCH_RATE, 0) AS CARRY_FORWARD_GLOBAL3_AMT,
  
  -- Additional Flags
  FS.FINANCIAL_GL_FLG,
  FS.BUDGETARY_CONTROL_FLG,
  FS.BALANCE_ID,
  
  -- Project and Program Segment WIDs
  COALESCE(LKP_SEG_PROJECT.ROW_WID, 0) AS SEG_PROJECT_WID,
  COALESCE(LKP_SEG_PROGRAM.ROW_WID, 0) AS SEG_PROGRAM_WID,
  
  -- Budget Dimension WIDs
  COALESCE(LKP_BUDGET_LEDGER.ROW_WID, 0) AS BUDGET_LEDGER_WID,
  COALESCE(LKP_BUDGET_MCAL_CAL.ROW_WID, 0) AS BUDGET_MCAL_CAL_WID,
  COALESCE(LKP_BUDGET.ROW_WID, 0) AS BUDGET_WID,
  COALESCE(LKP_PERIOD.MCAL_PERIOD_END_DT_WID, 0) AS BALANCE_DT_KEY,
  COALESCE(LKP_BUDGET_ACCOUNT.ROW_WID, 0) AS BUDGET_ACCOUNT_WID,
  
  -- Budget Segment WIDs (1-10) from W_BUDGET_SEGMENT_D  
  COALESCE(LKP_BUDGET_SEG1.ROW_WID, 0) AS BUDGET_SEGMENT1_WID,
  COALESCE(LKP_BUDGET_SEG2.ROW_WID, 0) AS BUDGET_SEGMENT2_WID,
  COALESCE(LKP_BUDGET_SEG3.ROW_WID, 0) AS BUDGET_SEGMENT3_WID,
  COALESCE(LKP_BUDGET_SEG4.ROW_WID, 0) AS BUDGET_SEGMENT4_WID,
  COALESCE(LKP_BUDGET_SEG5.ROW_WID, 0) AS BUDGET_SEGMENT5_WID,
  COALESCE(LKP_BUDGET_SEG6.ROW_WID, 0) AS BUDGET_SEGMENT6_WID,
  COALESCE(LKP_BUDGET_SEG7.ROW_WID, 0) AS BUDGET_SEGMENT7_WID,
  COALESCE(LKP_BUDGET_SEG8.ROW_WID, 0) AS BUDGET_SEGMENT8_WID,
  COALESCE(LKP_BUDGET_SEG9.ROW_WID, 0) AS BUDGET_SEGMENT9_WID,
  COALESCE(LKP_BUDGET_SEG10.ROW_WID, 0) AS BUDGET_SEGMENT10_WID,
  
  -- X Segment Codes (1-9) from W_GL_ACCOUNT_D
  COALESCE(ACCT.ACCOUNT_SEG1_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT1_CODE,
  COALESCE(ACCT.ACCOUNT_SEG2_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT2_CODE,
  COALESCE(ACCT.ACCOUNT_SEG3_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT3_CODE,
  COALESCE(ACCT.ACCOUNT_SEG4_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT4_CODE,
  COALESCE(ACCT.ACCOUNT_SEG5_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT5_CODE,
  COALESCE(ACCT.ACCOUNT_SEG6_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT6_CODE,
  COALESCE(ACCT.ACCOUNT_SEG7_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT7_CODE,
  COALESCE(ACCT.ACCOUNT_SEG8_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT8_CODE,
  COALESCE(ACCT.ACCOUNT_SEG9_CODE, '__NOT_APPLICABLE__') AS X_SEGMENT9_CODE,
  
  -- Time Surrogate Keys (derived from BALANCE_DATE)
  CAST(DATE_FORMAT(FS.BALANCE_DATE, 'yyyy') AS INT) AS X_TS_YEAR_WID,
  CAST(DATE_FORMAT(FS.BALANCE_DATE, 'yyyyMM') AS INT) AS X_TS_PERIOD_WID,
  CAST(DATE_FORMAT(FS.BALANCE_DATE, 'yyyyMMdd') AS INT) AS X_TS_DATE_WID,
  
  -- Change Detection Indicator (OUTER strategy with timestamp comparison)
  CASE
    WHEN T.INTEGRATION_ID IS NOT NULL
      AND COALESCE(T.CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(FS.CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
      AND COALESCE(T.AUX1_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(FS.AUX1_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
      AND COALESCE(T.AUX2_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(FS.AUX2_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
      AND COALESCE(T.AUX3_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(FS.AUX3_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
      AND COALESCE(T.AUX4_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP)) = COALESCE(FS.AUX4_CHANGED_ON_DT, CAST('3714-01-01' AS TIMESTAMP))
    THEN 'N'  -- No change
    WHEN T.INTEGRATION_ID IS NOT NULL THEN 'U'  -- Update
    ELSE 'I'  -- Insert
  END AS IND_UPDATE

FROM workspace.oracle_edw.W_GL_BALANCE_FS FS

-- Main Joins to Account and Budget Dimensions
LEFT OUTER JOIN workspace.oracle_edw.W_GL_ACCOUNT_D ACCT
  ON FS.DATASOURCE_NUM_ID = ACCT.DATASOURCE_NUM_ID
  AND FS.GL_ACCOUNT_ID = ACCT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= ACCT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < ACCT.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_ACCOUNT_D BUDGET_ACCT
  ON FS.DATASOURCE_NUM_ID = BUDGET_ACCT.DATASOURCE_NUM_ID
  AND FS.BUDGET_ACCOUNT_ID = BUDGET_ACCT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= BUDGET_ACCT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < BUDGET_ACCT.EFFECTIVE_TO_DT

-- Exchange Rate Lookup for Global Currency Conversions
LEFT OUTER JOIN workspace.oracle_edw.W_GLOBAL_EXCH_RATE_G EXCH
  ON FS.LOC_CURR_CODE = EXCH.W_FROM_CURRENCY_CODE
  AND TRUNC(FS.BALANCE_DATE) = TRUNC(EXCH.EXCH_DATE)
  AND FS.DATASOURCE_NUM_ID = EXCH.DATASOURCE_NUM_ID
  AND FS.TENANT_ID = EXCH.TENANT_ID

-- Core Dimension Lookups
LEFT OUTER JOIN workspace.oracle_edw.W_LEDGER_D LKP_LEDGER
  ON FS.DATASOURCE_NUM_ID = LKP_LEDGER.DATASOURCE_NUM_ID
  AND FS.LEDGER_ID = LKP_LEDGER.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_COST_CENTER_D LKP_COST_CENTER
  ON FS.DATASOURCE_NUM_ID = LKP_COST_CENTER.DATASOURCE_NUM_ID
  AND COALESCE(FS.COST_CENTER_ID, CONCAT(FS.COST_CENTER_ATTRIB, '~', FS.COST_CENTER_NUM)) = LKP_COST_CENTER.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_COST_CENTER.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_COST_CENTER.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_BALANCING_SEGMENT_D LKP_BALANCING_SEG
  ON FS.DATASOURCE_NUM_ID = LKP_BALANCING_SEG.DATASOURCE_NUM_ID
  AND COALESCE(FS.BALANCING_SEGMENT_ID, CONCAT(ACCT.BALANCING_SEGMENT_ATTRIB, '~', ACCT.BALANCING_SEGMENT_NUM)) = LKP_BALANCING_SEG.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_BALANCING_SEG.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_BALANCING_SEG.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_NATURAL_ACCOUNT_D LKP_NATURAL_ACCT
  ON FS.DATASOURCE_NUM_ID = LKP_NATURAL_ACCT.DATASOURCE_NUM_ID
  AND COALESCE(FS.NATURAL_ACCOUNT_ID, CONCAT(ACCT.NATURAL_ACCOUNT_ATTRIB, '~', ACCT.NATURAL_ACCOUNT_NUM)) = LKP_NATURAL_ACCT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_NATURAL_ACCT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_NATURAL_ACCT.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_INT_ORG_D LKP_COMPANY_ORG
  ON FS.DATASOURCE_NUM_ID = LKP_COMPANY_ORG.DATASOURCE_NUM_ID
  AND FS.COMPANY_ORG_ID = LKP_COMPANY_ORG.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_INT_ORG_D LKP_BUSN_AREA_ORG
  ON FS.DATASOURCE_NUM_ID = LKP_BUSN_AREA_ORG.DATASOURCE_NUM_ID
  AND FS.BUSN_AREA_ORG_ID = LKP_BUSN_AREA_ORG.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_GL_ACCOUNT_D LKP_GL_ACCOUNT
  ON FS.DATASOURCE_NUM_ID = LKP_GL_ACCOUNT.DATASOURCE_NUM_ID
  AND FS.GL_ACCOUNT_ID = LKP_GL_ACCOUNT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_ACCOUNT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_ACCOUNT.EFFECTIVE_TO_DT

-- Date/Time Dimension Lookups
LEFT OUTER JOIN workspace.oracle_edw.W_DAY_D LKP_BALANCE_DT
  ON CAST(FS.BALANCE_DATE AS DATE) = LKP_BALANCE_DT.CAL_DAY

LEFT OUTER JOIN workspace.oracle_edw.W_HOUR_D LKP_BALANCE_TM
  ON CAST(DATE_FORMAT(FS.BALANCE_DATE, 'HH') AS INT) = LKP_BALANCE_TM.HOUR_NUM

LEFT OUTER JOIN workspace.oracle_edw.W_MCAL_PERIOD_D LKP_PERIOD
  ON FS.DATASOURCE_NUM_ID = LKP_PERIOD.DATASOURCE_NUM_ID
  AND FS.PERIOD_ID = LKP_PERIOD.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_MCAL_CAL_D LKP_MCAL_CAL
  ON FS.DATASOURCE_NUM_ID = LKP_MCAL_CAL.DATASOURCE_NUM_ID
  AND FS.MCAL_CAL_ID = LKP_MCAL_CAL.INTEGRATION_ID

-- Other Dimension Lookups
LEFT OUTER JOIN workspace.oracle_edw.W_TREASURY_SYMBOL_D LKP_TREASURY_SYMBOL
  ON FS.DATASOURCE_NUM_ID = LKP_TREASURY_SYMBOL.DATASOURCE_NUM_ID
  AND FS.TREASURY_SYMBOL_ID = LKP_TREASURY_SYMBOL.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_GL_DATASET_SECURITY_D LKP_GL_DATASET_SEC
  ON FS.DATASOURCE_NUM_ID = LKP_GL_DATASET_SEC.DATASOURCE_NUM_ID
  AND FS.GL_DATASET_SEC_ID = LKP_GL_DATASET_SEC.INTEGRATION_ID

-- GL Segment Lookups (1-10)
LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG1
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG1.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG1_INTEGRATION_ID = LKP_GL_SEG1.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG1.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG1.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG2
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG2.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG2_INTEGRATION_ID = LKP_GL_SEG2.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG2.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG2.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG3
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG3.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG3_INTEGRATION_ID = LKP_GL_SEG3.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG3.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG3.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG4
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG4.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG4_INTEGRATION_ID = LKP_GL_SEG4.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG4.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG4.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG5
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG5.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG5_INTEGRATION_ID = LKP_GL_SEG5.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG5.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG5.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG6
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG6.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG6_INTEGRATION_ID = LKP_GL_SEG6.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG6.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG6.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG7
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG7.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG7_INTEGRATION_ID = LKP_GL_SEG7.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG7.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG7.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG8
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG8.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG8_INTEGRATION_ID = LKP_GL_SEG8.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG8.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG8.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG9
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG9.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG9_INTEGRATION_ID = LKP_GL_SEG9.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG9.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG9.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_GL_SEG10
  ON FS.DATASOURCE_NUM_ID = LKP_GL_SEG10.DATASOURCE_NUM_ID
  AND ACCT.ACCOUNT_SEG10_INTEGRATION_ID = LKP_GL_SEG10.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_GL_SEG10.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_GL_SEG10.EFFECTIVE_TO_DT

-- Budget Related Lookups
LEFT OUTER JOIN workspace.oracle_edw.W_MCAL_PERIOD_D LKP_BUDGET_PERIOD
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_PERIOD.DATASOURCE_NUM_ID
  AND FS.BUDGET_PERIOD_ID = LKP_BUDGET_PERIOD.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BALANCE_TYPE_D LKP_BALANCE_TYPE
  ON FS.DATASOURCE_NUM_ID = LKP_BALANCE_TYPE.DATASOURCE_NUM_ID
  AND FS.BALANCE_TYPE_ID = LKP_BALANCE_TYPE.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_SEG_PROJECT
  ON FS.DATASOURCE_NUM_ID = LKP_SEG_PROJECT.DATASOURCE_NUM_ID
  AND ACCT.PROJECT_INTEGRATION_ID = LKP_SEG_PROJECT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_SEG_PROJECT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_SEG_PROJECT.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_GL_SEGMENT_D LKP_SEG_PROGRAM
  ON FS.DATASOURCE_NUM_ID = LKP_SEG_PROGRAM.DATASOURCE_NUM_ID
  AND ACCT.PROGRAM_INTEGRATION_ID = LKP_SEG_PROGRAM.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_SEG_PROGRAM.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_SEG_PROGRAM.EFFECTIVE_TO_DT

LEFT OUTER JOIN workspace.oracle_edw.W_LEDGER_D LKP_BUDGET_LEDGER
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_LEDGER.DATASOURCE_NUM_ID
  AND FS.BUDGET_LEDGER_ID = LKP_BUDGET_LEDGER.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_MCAL_CAL_D LKP_BUDGET_MCAL_CAL
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_MCAL_CAL.DATASOURCE_NUM_ID
  AND FS.BUDGET_MCAL_CAL_ID = LKP_BUDGET_MCAL_CAL.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_D LKP_BUDGET
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET.DATASOURCE_NUM_ID
  AND FS.BUDGET_ID = LKP_BUDGET.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_ACCOUNT_D LKP_BUDGET_ACCOUNT
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_ACCOUNT.DATASOURCE_NUM_ID
  AND FS.BUDGET_ACCOUNT_ID = LKP_BUDGET_ACCOUNT.INTEGRATION_ID
  AND FS.BALANCE_DATE >= LKP_BUDGET_ACCOUNT.EFFECTIVE_FROM_DT
  AND FS.BALANCE_DATE < LKP_BUDGET_ACCOUNT.EFFECTIVE_TO_DT

-- Budget Segment Lookups (1-10)
LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG1
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG1.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG1_INTEGRATION_ID = LKP_BUDGET_SEG1.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG2
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG2.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG2_INTEGRATION_ID = LKP_BUDGET_SEG2.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG3
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG3.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG3_INTEGRATION_ID = LKP_BUDGET_SEG3.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG4
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG4.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG4_INTEGRATION_ID = LKP_BUDGET_SEG4.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG5
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG5.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG5_INTEGRATION_ID = LKP_BUDGET_SEG5.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG6
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG6.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG6_INTEGRATION_ID = LKP_BUDGET_SEG6.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG7
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG7.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG7_INTEGRATION_ID = LKP_BUDGET_SEG7.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG8
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG8.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG8_INTEGRATION_ID = LKP_BUDGET_SEG8.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG9
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG9.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG9_INTEGRATION_ID = LKP_BUDGET_SEG9.INTEGRATION_ID

LEFT OUTER JOIN workspace.oracle_edw.W_BUDGET_SEGMENT_D LKP_BUDGET_SEG10
  ON FS.DATASOURCE_NUM_ID = LKP_BUDGET_SEG10.DATASOURCE_NUM_ID
  AND BUDGET_ACCT.ACCOUNT_SEG10_INTEGRATION_ID = LKP_BUDGET_SEG10.INTEGRATION_ID

-- LEFT OUTER JOIN to target table for change detection (OUTER strategy)
LEFT OUTER JOIN workspace.oracle_edw.W_GL_BALANCE_F T
  ON FS.INTEGRATION_ID = T.INTEGRATION_ID
  AND FS.DATASOURCE_NUM_ID = T.DATASOURCE_NUM_ID

In [ ]:
%sql
-- Step 2: Update existing rows using MERGE (for IND_UPDATE = 'U')
MERGE INTO workspace.oracle_edw.W_GL_BALANCE_F AS T
USING (
  SELECT * FROM I$_2625555_5 WHERE IND_UPDATE = 'U'
) S
ON T.INTEGRATION_ID = S.INTEGRATION_ID
  AND T.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
WHEN MATCHED THEN
  UPDATE SET
    T.LEDGER_WID = S.LEDGER_WID,
    T.COST_CENTER_WID = S.COST_CENTER_WID,
    T.BALANCING_SEGMENT_WID = S.BALANCING_SEGMENT_WID,
    T.NATURAL_ACCOUNT_WID = S.NATURAL_ACCOUNT_WID,
    T.COMPANY_ORG_WID = S.COMPANY_ORG_WID,
    T.BUSN_AREA_ORG_WID = S.BUSN_AREA_ORG_WID,
    T.GL_ACCOUNT_WID = S.GL_ACCOUNT_WID,
    T.BALANCE_DT_WID = S.BALANCE_DT_WID,
    T.BALANCE_TM_WID = S.BALANCE_TM_WID,
    T.TREASURY_SYMBOL_WID = S.TREASURY_SYMBOL_WID,
    T.MCAL_CAL_WID = S.MCAL_CAL_WID,
    T.GL_DATASET_SEC_WID = S.GL_DATASET_SEC_WID,
    T.DB_CR_IND = S.DB_CR_IND,
    T.BALANCE_ACCT_AMT = S.BALANCE_ACCT_AMT,
    T.BALANCE_LOC_AMT = S.BALANCE_LOC_AMT,
    T.BALANCE_GLOBAL1_AMT = S.BALANCE_GLOBAL1_AMT,
    T.BALANCE_GLOBAL2_AMT = S.BALANCE_GLOBAL2_AMT,
    T.BALANCE_GLOBAL3_AMT = S.BALANCE_GLOBAL3_AMT,
    T.ACTIVITY_ACCT_AMT = S.ACTIVITY_ACCT_AMT,
    T.ACTIVITY_LOC_AMT = S.ACTIVITY_LOC_AMT,
    T.ACTIVITY_GLOBAL1_AMT = S.ACTIVITY_GLOBAL1_AMT,
    T.ACTIVITY_GLOBAL2_AMT = S.ACTIVITY_GLOBAL2_AMT,
    T.ACTIVITY_GLOBAL3_AMT = S.ACTIVITY_GLOBAL3_AMT,
    T.ACCT_CURR_CODE = S.ACCT_CURR_CODE,
    T.LOC_CURR_CODE = S.LOC_CURR_CODE,
    T.TENANT_ID = S.TENANT_ID,
    T.X_CUSTOM = S.X_CUSTOM,
    T.BALANCE_TYPE_FLAG = S.BALANCE_TYPE_FLAG,
    T.TRANSLATED_FLAG = S.TRANSLATED_FLAG,
    T.SUMMARY_ACCOUNT_FLAG = S.SUMMARY_ACCOUNT_FLAG,
    T.CREATED_ON_DT = CAST(S.CREATED_ON_DT AS TIMESTAMP),
    T.CHANGED_ON_DT = CAST(S.CHANGED_ON_DT AS TIMESTAMP),
    T.AUX1_CHANGED_ON_DT = CAST(S.AUX1_CHANGED_ON_DT AS TIMESTAMP),
    T.AUX2_CHANGED_ON_DT = CAST(S.AUX2_CHANGED_ON_DT AS TIMESTAMP),
    T.AUX3_CHANGED_ON_DT = CAST(S.AUX3_CHANGED_ON_DT AS TIMESTAMP),
    T.AUX4_CHANGED_ON_DT = CAST(S.AUX4_CHANGED_ON_DT AS TIMESTAMP),
    T.GL_SEGMENT1_WID = S.GL_SEGMENT1_WID,
    T.GL_SEGMENT2_WID = S.GL_SEGMENT2_WID,
    T.GL_SEGMENT3_WID = S.GL_SEGMENT3_WID,
    T.GL_SEGMENT4_WID = S.GL_SEGMENT4_WID,
    T.GL_SEGMENT5_WID = S.GL_SEGMENT5_WID,
    T.GL_SEGMENT6_WID = S.GL_SEGMENT6_WID,
    T.GL_SEGMENT7_WID = S.GL_SEGMENT7_WID,
    T.GL_SEGMENT8_WID = S.GL_SEGMENT8_WID,
    T.GL_SEGMENT9_WID = S.GL_SEGMENT9_WID,
    T.GL_SEGMENT10_WID = S.GL_SEGMENT10_WID,
    T.BUDGET_PERIOD_WID = S.BUDGET_PERIOD_WID,
    T.BALANCE_TYPE_WID = S.BALANCE_TYPE_WID,
    T.CARRY_FORWARD_ACCT_AMT = S.CARRY_FORWARD_ACCT_AMT,
    T.CARRY_FORWARD_LOC_AMT = S.CARRY_FORWARD_LOC_AMT,
    T.CARRY_FORWARD_GLOBAL1_AMT = S.CARRY_FORWARD_GLOBAL1_AMT,
    T.CARRY_FORWARD_GLOBAL2_AMT = S.CARRY_FORWARD_GLOBAL2_AMT,
    T.CARRY_FORWARD_GLOBAL3_AMT = S.CARRY_FORWARD_GLOBAL3_AMT,
    T.FINANCIAL_GL_FLG = S.FINANCIAL_GL_FLG,
    T.BUDGETARY_CONTROL_FLG = S.BUDGETARY_CONTROL_FLG,
    T.BALANCE_ID = S.BALANCE_ID,
    T.SEG_PROJECT_WID = S.SEG_PROJECT_WID,
    T.SEG_PROGRAM_WID = S.SEG_PROGRAM_WID,
    T.BUDGET_LEDGER_WID = S.BUDGET_LEDGER_WID,
    T.BUDGET_MCAL_CAL_WID = S.BUDGET_MCAL_CAL_WID,
    T.BUDGET_WID = S.BUDGET_WID,
    T.BALANCE_DT_KEY = S.BALANCE_DT_KEY,
    T.BUDGET_ACCOUNT_WID = S.BUDGET_ACCOUNT_WID,
    T.BUDGET_SEGMENT1_WID = S.BUDGET_SEGMENT1_WID,
    T.BUDGET_SEGMENT2_WID = S.BUDGET_SEGMENT2_WID,
    T.BUDGET_SEGMENT3_WID = S.BUDGET_SEGMENT3_WID,
    T.BUDGET_SEGMENT4_WID = S.BUDGET_SEGMENT4_WID,
    T.BUDGET_SEGMENT5_WID = S.BUDGET_SEGMENT5_WID,
    T.BUDGET_SEGMENT6_WID = S.BUDGET_SEGMENT6_WID,
    T.BUDGET_SEGMENT7_WID = S.BUDGET_SEGMENT7_WID,
    T.BUDGET_SEGMENT8_WID = S.BUDGET_SEGMENT8_WID,
    T.BUDGET_SEGMENT9_WID = S.BUDGET_SEGMENT9_WID,
    T.BUDGET_SEGMENT10_WID = S.BUDGET_SEGMENT10_WID,
    T.X_SEGMENT1_CODE = S.X_SEGMENT1_CODE,
    T.X_SEGMENT2_CODE = S.X_SEGMENT2_CODE,
    T.X_SEGMENT3_CODE = S.X_SEGMENT3_CODE,
    T.X_SEGMENT4_CODE = S.X_SEGMENT4_CODE,
    T.X_SEGMENT5_CODE = S.X_SEGMENT5_CODE,
    T.X_SEGMENT6_CODE = S.X_SEGMENT6_CODE,
    T.X_SEGMENT7_CODE = S.X_SEGMENT7_CODE,
    T.X_SEGMENT8_CODE = S.X_SEGMENT8_CODE,
    T.X_SEGMENT9_CODE = S.X_SEGMENT9_CODE,
    T.X_TS_YEAR_WID = S.X_TS_YEAR_WID,
    T.X_TS_PERIOD_WID = S.X_TS_PERIOD_WID,
    T.X_TS_DATE_WID = S.X_TS_DATE_WID,
    T.ETL_PROC_WID = 2625555

In [ ]:
%sql
-- Step 3: Insert new rows (for IND_UPDATE = 'I')
INSERT INTO workspace.oracle_edw.W_GL_BALANCE_F
SELECT
  LEDGER_WID,
  COST_CENTER_WID,
  BALANCING_SEGMENT_WID,
  NATURAL_ACCOUNT_WID,
  COMPANY_ORG_WID,
  BUSN_AREA_ORG_WID,
  GL_ACCOUNT_WID,
  BALANCE_DT_WID,
  BALANCE_TM_WID,
  TREASURY_SYMBOL_WID,
  MCAL_CAL_WID,
  GL_DATASET_SEC_WID,
  DB_CR_IND,
  BALANCE_ACCT_AMT,
  BALANCE_LOC_AMT,
  BALANCE_GLOBAL1_AMT,
  BALANCE_GLOBAL2_AMT,
  BALANCE_GLOBAL3_AMT,
  ACTIVITY_ACCT_AMT,
  ACTIVITY_LOC_AMT,
  ACTIVITY_GLOBAL1_AMT,
  ACTIVITY_GLOBAL2_AMT,
  ACTIVITY_GLOBAL3_AMT,
  ACCT_CURR_CODE,
  LOC_CURR_CODE,
  INTEGRATION_ID,
  DATASOURCE_NUM_ID,
  TENANT_ID,
  X_CUSTOM,
  BALANCE_TYPE_FLAG,
  TRANSLATED_FLAG,
  SUMMARY_ACCOUNT_FLAG,
  CAST(CREATED_ON_DT AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(AUX1_CHANGED_ON_DT AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  CAST(AUX2_CHANGED_ON_DT AS TIMESTAMP) AS AUX2_CHANGED_ON_DT,
  CAST(AUX3_CHANGED_ON_DT AS TIMESTAMP) AS AUX3_CHANGED_ON_DT,
  CAST(AUX4_CHANGED_ON_DT AS TIMESTAMP) AS AUX4_CHANGED_ON_DT,
  GL_SEGMENT1_WID,
  GL_SEGMENT2_WID,
  GL_SEGMENT3_WID,
  GL_SEGMENT4_WID,
  GL_SEGMENT5_WID,
  GL_SEGMENT6_WID,
  GL_SEGMENT7_WID,
  GL_SEGMENT8_WID,
  GL_SEGMENT9_WID,
  GL_SEGMENT10_WID,
  BUDGET_PERIOD_WID,
  BALANCE_TYPE_WID,
  CARRY_FORWARD_ACCT_AMT,
  CARRY_FORWARD_LOC_AMT,
  CARRY_FORWARD_GLOBAL1_AMT,
  CARRY_FORWARD_GLOBAL2_AMT,
  CARRY_FORWARD_GLOBAL3_AMT,
  FINANCIAL_GL_FLG,
  BUDGETARY_CONTROL_FLG,
  BALANCE_ID,
  SEG_PROJECT_WID,
  SEG_PROGRAM_WID,
  BUDGET_LEDGER_WID,
  BUDGET_MCAL_CAL_WID,
  BUDGET_WID,
  BALANCE_DT_KEY,
  BUDGET_ACCOUNT_WID,
  BUDGET_SEGMENT1_WID,
  BUDGET_SEGMENT2_WID,
  BUDGET_SEGMENT3_WID,
  BUDGET_SEGMENT4_WID,
  BUDGET_SEGMENT5_WID,
  BUDGET_SEGMENT6_WID,
  BUDGET_SEGMENT7_WID,
  BUDGET_SEGMENT8_WID,
  BUDGET_SEGMENT9_WID,
  BUDGET_SEGMENT10_WID,
  X_SEGMENT1_CODE,
  X_SEGMENT2_CODE,
  X_SEGMENT3_CODE,
  X_SEGMENT4_CODE,
  X_SEGMENT5_CODE,
  X_SEGMENT6_CODE,
  X_SEGMENT7_CODE,
  X_SEGMENT8_CODE,
  X_SEGMENT9_CODE,
  X_TS_YEAR_WID,
  X_TS_PERIOD_WID,
  X_TS_DATE_WID,
  2625555 AS ETL_PROC_WID
FROM I$_2625555_5
WHERE IND_UPDATE = 'I'

In [ ]:
%sql
-- Step 4: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_GLBALANCEFACT' AS PACKAGE_NAME,
    'W_GL_BALANCE_F' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625555 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    '1' AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 5: Insert to W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2625555 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_GLBALANCEFACT'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 6: Drop flow table
DROP VIEW IF EXISTS I$_2625555_5